# Predictions
### Steps : 
* Train multiple models on the dataset  
* Evaluate them  
* Select the best one 

## Importing dataset and libraries

In [5]:
import pandas as pd
### Preprocessing libraries
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
### Models
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
df = pd.read_csv('../assets/data/clean_data.csv')

## Prepare data for the training

### Split X and y

In [3]:
X = df.drop(["Div","Date","FTR", "FTHG", "FTAG", "HTHG", "HTAG", "HTR", "HS", "AS", "AST", "HF", "AF", "HC",'AC', 'HY', 'AY', 'HR', 'AR', "Numerical_ftr", "Numerical_htr"], axis=1)
y = df["FTR"]

Establish categorical columns and numerical columns

In [4]:
categorical_features = [col for col in X.select_dtypes(include='object')]
numerical_features = [col for col in X.select_dtypes(include='float64')]

Instantiate preprocessing pipelines

In [ ]:
num_pipeline = make_pipeline(SimpleImputer(strategy="median"), StandardScaler())
cat_pipeline = make_pipeline(SimpleImputer(strategy="most_frequent"), OneHotEncoder(handle_unknown="ignore"))
full_pipeline = make_column_transformer((num_pipeline, numerical_features), (cat_pipeline, categorical_features))

Pass the data trough the pipelines

In [ ]:
X = full_pipeline.fit_transform(X)
encoder = OneHotEncoder(handle_unknown="ignore")

Split train/test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)